In [1]:
import sys
import random
import PySide6
from PySide6 import QtCore, QtWidgets, QtGui
from PySide6.QtWidgets import *
from PySide6.QtCore import Qt
from PySide6.QtGui import QPalette, QColor
import numpy as np
import pandas as pd


In [2]:
data=None

try:
    data=pd.read_feather("data/stage1_partial.feather")
except:
    try:
        print("Loading backup, main data corrupted.")
        data=pd.read_feather("data/stage1_partial.backup.feather")
    except:
        print("Loading stage 0, main data & backup corrupted.")
        data=pd.read_feather("data/stage0.feather")


In [3]:
(1-data[data["volcano_certainty"]==-1].shape[0]/data.shape[0])*100

0.6740347920900058

In [4]:
data[data["volcano_certainty"]==-1].head()

,path,volcano_certainty,day_night,has_fume,is_explosion
136,images\2000\dic\p1216001.jpg,-1,-1,-1,-1
137,images\2000\dic\p1217001.jpg,-1,-1,-1,-1
138,images\2000\dic\p1217002.jpg,-1,-1,-1,-1
139,images\2000\dic\p1217005.jpg,-1,-1,-1,-1
140,images\2000\dic\p1217006.jpg,-1,-1,-1,-1


In [5]:
def save(is_backup=False):
    data.to_feather(f"data/stage1_partial{'.backup' if is_backup else ''}.feather")
save(True)

In [6]:
data[data["volcano_certainty"]==-1].index

Index([  136,   137,   138,   139,   140,   141,   142,   143,   144,   145,
       ...
       20167, 20168, 20169, 20170, 20171, 20172, 20173, 20174, 20175, 20176],
      dtype='int64', length=20041)

In [7]:
def get_next():
    idx=data[data["volcano_certainty"]==-1].index
    if(idx.shape[0]==0):
        return None, 0, None
    return idx[0],idx.shape[0], data.iloc[idx[0]]

In [8]:
next_idx,remainder,row=get_next()

In [9]:
def safe_update(idx, values):
    save(True)
    data.iloc[idx,1:]=values
    save(False)

In [10]:
keys = ["volcano_certainty", "day_night", "has_fume", "is_explosion"]
labels = {
    keys[0]:"No volcán",
    keys[1]:"Noche",
    keys[2]:"Sin fumarola",
    keys[3]:"No explotando",
}
endLabels = {
    keys[0]:"Es volcán",
    keys[1]:"Día",
    keys[2]:"Tiene fumarola",
    keys[3]:"Está explotando",
}

In [11]:
window=None

In [12]:
class ImageDisplay(QWidget):

    def loadpixmap(self, path):
        self.pixmap=QtGui.QPixmap(path)#.scaledToHeight(self.geometry().width())
        self.label.setPixmap(self.pixmap)
        
    def __init__(self, path = ""):
        super().__init__()
        label=QLabel()
        label.setAlignment(Qt.Alignment.AlignCenter)
        label.setScaledContents(True)
        self.label=label
        self.loadpixmap(path)
        
        self.layout=QGridLayout()
        self.layout.addWidget(label, 0,0)
        self.setLayout(self.layout)

In [27]:
class DataLayout(QWidget):

    @QtCore.Slot()
    def value_changed(self):
        self.values=[]
        for slider in self.sliders:
            self.values.append(slider.value())
        
    @QtCore.Slot()
    def button_clicked(self):
        global next_idx,remainder,row
        safe_update(next_idx, self.values)
        next_idx,remainder,row=get_next()
        window.image.loadpixmap(row["path"])
        #for slider in self.sliders:
        #    slider.setValue(0)
        self.values=[]
        for slider in self.sliders:
            self.values.append(slider.value())
        rem=data[data["volcano_certainty"]==-1].shape[0]
        self.progress.setText(f"Remaining: {rem}")
        #window.close()
        
    
    def __init__(self):
        super().__init__()
        self.setMinimumWidth(600)
        self.setMaximumWidth(600)
        layout=QGridLayout()

        self.sliders=[]
        self.values=[0 for k in keys]
        rem=data[data["volcano_certainty"]==-1].shape[0]
        self.progress = QLabel(f"Remaining: {rem}")
        self.progress.setMaximumHeight(50)
        self.progress.setAlignment(Qt.Alignment.AlignCenter)
        layout.addWidget(self.progress,0,0,1,4)
        
        for i,key in enumerate(keys):
            slider=QSlider(Qt.Orientation.Horizontal)
            slider.setMinimumHeight(50)
            if(key=="volcano_certainty"):
                slider.setSingleStep(10)
                slider.setPageStep(10)
            elif(key=="day_night"):
                slider.setSingleStep(5)
                slider.setPageStep(5)
            else:
                slider.setSingleStep(2)
                slider.setPageStep(5)
            slider.setMinimum(0)
            slider.setMaximum(10)
            slider.setTickPosition(QSlider.TicksBelow)
            slider.setSizePolicy(QSizePolicy.MinimumExpanding,QSizePolicy.Fixed)
            slider.valueChanged.connect(self.value_changed)
            self.sliders.append(slider)
            
            label=QLabel(labels[key])
            label.setMinimumHeight(50)
            label.setAlignment(Qt.Alignment.AlignCenter)
            
            label2=QLabel(endLabels[key])
            label2.setMinimumHeight(50)
            label2.setAlignment(Qt.Alignment.AlignCenter)
            
            layout.addWidget(label, 1+i,0)
            layout.addWidget(slider, 1+i,1,1,4)
            layout.addWidget(label2, 1+i,5)

        button=QPushButton("Siguiente")
        button.setMinimumHeight(50)
        #button.setAlignment(Qt.Alignment.AlignCenter)
        button.clicked.connect(self.button_clicked)
        layout.addWidget(button,len(keys)+1,1, 1,4)
        
        #layout.addRow(label1,slider1)
        #slider2=QSlider(Qt.Orientation.Horizontal)
        #layout.addRow("TEST",slider2)
        self.setLayout(layout)

In [28]:

class KeyPressFilter(QtCore.QObject):

    def eventFilter(self, widget, event):
        if event.type() == QtCore.QEvent.KeyPress:
            key = event.keyCombination().key()
            if key==Qt.Key.Key_Return:
                window.data.button_clicked()
        return False

In [29]:
class MainWindow(QMainWindow):

    def __init__(self):
        super(MainWindow, self).__init__()

        self.setWindowTitle("Classifier")

        layout = QHBoxLayout()
        left = ImageDisplay(row["path"])
        right= DataLayout()

        self.data=right
        self.image=left

        layout.addWidget(left)
        layout.addWidget(right)

        widget = QWidget()
        widget.setLayout(layout)
        self.setCentralWidget(widget)
        self.eventFilter = KeyPressFilter(parent=self)
        self.installEventFilter(self.eventFilter)

In [30]:
if __name__ == "__main__" and remainder>0:
    if not QtWidgets.QApplication.instance():
        app = QtWidgets.QApplication([])
    else:
        app = QtWidgets.QApplication.instance()
    font=QtGui.QFont("Segoe UI", 14)
    app.setFont(font)
    app.setStyle(QStyleFactory.create("windows"))
    window = MainWindow()
    window.resize(1900,1000)
    window.showMaximized()

    app.exec()